# Recursive Functions in Lettuce


In this lecture, we will explore recursive functions in Lettuce. Thus far, we have defined functions in lettuce.
As an example, we can define a program with a function call.

~~~
let f = function (x)  x * x + 20 
   in 
     f(20)
~~~

The program above defined a function that takes in an input `x` and returns `x * x + 20`. Thus, the call
`f(20)` returns the value `420`.

### Why do we need special handling for recursion?

If we attempted to define a function such as factorial: 

~~~
let fact = function (x) 
               if (x <= 0) 
               then 1
               else x * fact(x-1)
  in 
    fact(20)
~~~

This will lead to a problem due to the way we have been handling the `let` binding.

Recall that to evaluate a let binding

~~~
let x = <def_expr> in <body_expr>
~~~

under an environment $\sigma$, we proceed in the following steps:
- Evaluate `<def_expr>` under the environment $\sigma$. Let $v$ be the value.
- Evaluate `<body_expr>` under the environment $\sigma[x \mapsto v]$.

The problem is that the identifier $x$ that is being defined is itself not in scope when we
are evaluatating `<def_expr>`. 

Apply this logic to the recursive `fact` function (recalled again below):

~~~
let fact = function (x) 
               if (x <= 0) 
               then 1
               else x * fact(x - 1)
  in 
    fact(10)
~~~

Notice that the identifier `fact` occurs in the `<def_expr>` part of the definition as well, where it is not going to be a part of the environment. Thus, our interpreter thus far will reject this program during runtime.
While evaluating the sub expression 

~~~
FunCall(Ident("fact"), Minus(Ident("x"), Const(1)))
~~~

The interpreter tries to look up the identifier `fact` and finds that it is not part of the environment.

### Handling Recursion

We will now discuss how to handle recursion properly. To enable this, we will need to treat recursive definitions different from regular definitions.

Thus in our new version, we will use a `let rec` construct that tells us that whatever we are going to define is going to be recursive. 

~~~
let rec fact = function (x) 
               if (x <= 0) 
               then 1
               else x * fact(x - 1) 

in 
    fact(10)
~~~

The `let rec` construct is very similar to a let binding but with one big difference: the defining expression for a let rec has to be a function. For instance:

~~~
let rec multiply_by_10 = function (x)
                            if (x == 0)
                            then 0
                            else 10 + multiply_by_10(x - 1)
~~~

We will not allow recursive definitions that are not functions. For instance: 

~~~
let rec x = 1 + x
~~~

makes no sense at all, and is disallowed.

Therefore, we will extend Lettuce with the construct

~~~
let rec <function_name_identifier> = function (<param_identifier>)
                                         function_body_expr
                                  in 
                         body_expr
~~~
                  


Let us extend the grammar of a stripped down version:


$$\begin{array}{rcll}
\mathbf{Program} & \rightarrow & TopLevel(\mathbf{Expr}) \\[5pt]
\mathbf{Expr} & \rightarrow & Const(\mathbf{Number}) \\
 & | & Ident(\mathbf{Identifier}) \\
 & | & Plus(\mathbf{Expr}, \mathbf{Expr}) \\
 & | & Mult(\mathbf{Expr}, \mathbf{Expr}) \\
 & | & Eq(\mathbf{Expr}, \mathbf{Expr}) \\
 & | & Geq (\mathbf{Expr}, \mathbf{Expr}) \\
 & | & IfThenElse(\mathbf{Expr}, \mathbf{Expr}, \mathbf{Expr}) & \text{if (expr) then expr else expr} \\
 & | & Let( \mathbf{Identifier}, \mathbf{Expr}, \mathbf{Expr}) & \text{let identifier = expr in expr} \\
 & | & FunDef( \mathbf{Identifier}, \mathbf{Expr}) & \text{function (identifier-formal-parameter) expr } \\ 
 & | & FunCall(\mathbf{Expr}, \mathbf{Expr}) & \text{function call - expr(expr)} \\
 & | & \color{red}{LetRec(\mathbf{Identifier}, \mathbf{Identifier}, \mathbf{Expr}, \mathbf{Expr}) } & \text{argument 1 - function name, argument 2 - parameter}\\
 &&& \text{argument 3 - function definition expression, argument 4 - body expr} \\[5pt]
\end{array}$$

For example, we would like to represent the concrete syntax

~~~
let rec f  = function (z) 
                if (0 >= z) then 1 else 1 + f(z - 1)
    in f(10)
~~~

as the following abstract syntax:

~~~
LetRec("f", "z", IfThenElse( 
                         Geq(Const(0), Ident("z")),
                         Const(1),
                         Plus(Const(1), FunCall(Ident("f"), Minus(Ident("z"), Const(1))))
                         ), 
                  FunCall(Ident("f"), Const(10))
         )
~~~



As an exercise, match various parts of the abstract syntax with the concrete syntax above.

In [ ]:
sealed trait Program
sealed trait Expr
case class Const(f: Double) extends Expr
case class Ident(s: String) extends Expr
case class Minus(e1: Expr, e2: Expr) extends Expr
case class Plus(e1: Expr, e2: Expr) extends Expr
case class Mult(e1: Expr, e2: Expr) extends Expr
case class Eq(e1: Expr, e2: Expr) extends Expr
case class Geq(e1: Expr, e2: Expr) extends Expr
case class IfThenElse(e1: Expr, e2: Expr, e3: Expr) extends Expr
case class Let(x: String, e1: Expr, e2: Expr) extends Expr
case class FunDef(id: String, e: Expr) extends Expr
case class FunCall(calledFun: Expr, argExpr: Expr) extends Expr
/* Here is the new stuff */
case class LetRec(funName: String, param: String, funExpr: Expr, bodyExpr: Expr) extends Expr

case class TopLevel(e: Expr) extends Program

In [ ]:
val p1 = TopLevel(LetRec("f", "z", IfThenElse( 
                         Geq(Const(0), Ident("z")),
                         Const(1),
                         Plus(Const(1), FunCall(Ident("f"), Minus(Ident("z"), Const(1))))
                         ), 
                  FunCall(Ident("f"), Const(10))
         ))

# Handling Recursion: Using a Y-Combinator.

We will explain an elegant approach to handling recursion using an idea called the Y-combinator. To be sure, it is
a theoretical idea but it's elegance is mind blowing, and therefore it is worth learning just for that.

There is a very nice and elementary description of Y-combinator using Lisp here: https://mvanier.livejournal.com/2897.html

However, it may require you to know a little bit of Lisp/Scheme.  We will cover the gist of this in Lettuce, and leave a deeper dive into the Y-Combinator to the curious ones in this class (which we hope is the entire class)

To begin with, let us just deal with the recursive version of factorial

~~~
let factorial = function (n) 
            if (n == 0) 
            then 1
            else n * factorial(n-1)
   in ...
~~~

This will not work in our current version of Lettuce because, when evaluating the body of the factorial function, we do not have factorial in our environment.

To make sure of this, we will cut and paste our original intepreter below and if ever we see a recursion, we will blow our top and throw an exception for now.



In [ ]:
/*-- CUT AND PASTE CODE FROM NOTEBOOK on Functions --*/
/* 1. Define the values */
sealed trait Value 
case class NumValue(d: Double) extends Value
case class BoolValue(b: Boolean) extends Value
/* -- Let us add Closure to the set of values --*/
case class Closure(x: String, e: Expr, pi: Map[String, Value]) extends Value
case object ErrorValue extends Value


/*2. Operators on values */

def valueToNumber(v: Value): Double = v match {
    case NumValue(d) => d
    case _ => throw new IllegalArgumentException(s"Error: Asking me to convert Value: $v to a number")
}

def valueToBoolean(v: Value): Boolean = v match {
    case BoolValue(b) => b
    case _ => throw new IllegalArgumentException(s"Error: Asking me to convert Value: $v to a boolean")
}

def valueToClosure(v: Value): Closure = v match {
    case Closure(x, e, pi) => Closure(x, e, pi)
    case _ =>  throw new IllegalArgumentException(s"Error: Asking me to convert Value: $v to a closure")
}


def evalExpr(e: Expr, env: Map[String, Value]): Value =  {
    
    /* Method to deal with binary arithmetic operations */
    
    def applyArith2 (e1: Expr, e2: Expr) (fun: (Double , Double) => Double) = {
        val v1 = valueToNumber(evalExpr(e1, env))
        val v2 = valueToNumber(evalExpr(e2, env))
        val v3 = fun(v1, v2)
        NumValue(v3)
    }  /* -- We have deliberately curried the method --*/
    
    /* Helper method to deal with unary arithmetic */
    def applyArith1(e: Expr) (fun: Double => Double) = {
        val v = valueToNumber(evalExpr(e, env))
        val v1 = fun(v)
        NumValue(v1)
    }
    
    /* Helper method to deal with comparison operators */
    def applyComp(e1: Expr, e2: Expr) (fun: (Double, Double) => Boolean) = {
        val v1 = valueToNumber(evalExpr(e1, env))
        val v2 = valueToNumber(evalExpr(e2, env))
        val v3 = fun(v1, v2)
        BoolValue(v3)
    }
    
   
    e match {
        case Const(f) => NumValue(f)
        
        case Ident(x) => {
            if (env contains x) 
                env(x)
            else 
                throw new IllegalArgumentException(s"Undefined identifier $x")
        }
    
       
    
        case Plus(e1, e2) => applyArith2 (e1, e2) ( _ + _ )
    
        case Minus(e1, e2) => applyArith2(e1, e2) ( _ - _ )
    
        case Mult(e1, e2) =>  applyArith2(e1, e2) (_ * _)
    
      
    
      
        case Geq(e1, e2) => applyComp(e1, e2)(_ >= _)
    
        case Eq(e1, e2) => applyComp(e1, e2)(_ == _)
    
        
        case IfThenElse(e1, e2, e3) => {
            val v = evalExpr(e1, env)
            v match {
                case BoolValue(true) => evalExpr(e2, env)
                case BoolValue(false) => evalExpr(e3, env)
                case _ => throw new IllegalArgumentException(s"If-then-else condition expr: ${e1} is non-boolean -- evaluates to ${v}")
            }
        }
    
        case Let(x, e1, e2) => {
            val v1 = evalExpr(e1, env)  // eval e1
            val env2 = env + (x -> v1) // create a new extended env
            evalExpr(e2, env2) // eval e2 under that.
        }
    
        case FunDef(x, e) => {
            Closure(x, e, env) // Return a closure with the current enviroment.
        }
        
        case FunCall(e1, e2) => {
            val v1 = evalExpr(e1, env)
            val v2 = evalExpr(e2, env)
            v1 match {
                case Closure(x, closure_ex, closed_env) => {
                    // First extend closed_env by binding x to v2
                    val new_env = closed_env + ( x -> v2)
                    // Evaluate the body of the closure under the extended environment.
                    evalExpr(closure_ex, new_env)
                }
                case _ => throw new IllegalArgumentException(s"Function call error: expression $e1 does not evaluate to a closure")
            }
        }
    }
}

def evalProgram(p: Program) = {
    val m: Map[String, Value] = Map[String,Value]()
    p match { 
        case TopLevel(e) => evalExpr(e, m)
    }
}


Let us naively try to write down the factorial function using `let` rather than `let rec`

~~~
let factorial = function (n) 
                 if (n == 0) 
                 then 1
                 else n * factorial(n - 1)
            in 
         factorial(10)
~~~

In [ ]:
/* -- 
let factorial = function (n) 
            if (n == 0) 
            then 1
            else n * factorial(n-1)
   in 
    factorial(10)
--*/

val fact = Ident("factorial")
val n = Ident("n")
// if n == 0 then 1 else n * factorial(n-1)
val ite = IfThenElse(Eq(n, Const(0)), Const(1), Mult(n, FunCall(fact, Minus(n, Const(1)))))
val fundef = FunDef("n", ite)
val e = Let("factorial", fundef, FunCall(fact, Const(10)))
val wrongFactorial = TopLevel(e)


Let us evaluate this program and see what goes wrong.

In [ ]:
evalProgram(wrongFactorial)

Notice how the attempt to evaluate the factorial program fails exactly as predicted. How do we fix this?

To do so, we can always make a non recursive version like this:

~~~
let almost_factorial = function (f) 
                         function (n) 
                            if (n == 0) 
                            then 1
                            else n * f(n-1) 
in 
.... 
~~~ 

The idea here is to have a curried function `almost_factorial` that takes in two arguments, an input function `f` and an argument `n`. The idea here is that if n == 0, then it returns 1, otherwise it calls `f` on `n-1`.

If `f` is any function that can safely compute a factorial, this would totally work. But we are left with a
chicken and egg question: how do we get such a function in the first place?


The key insight would be like this:

- If we could define `almost_factorial` and just pass it as the argument f of the function, will it work?

It would except that we will have a type error. 

- `almost_factorial` needs two arguments but the function f being passed in takes only one argument. 

Suppose we did the following minor change

~~~
let almost_factorial = function (f) 
                          function (n) 
                              if (n == 0)
                              then 1 
                              else n * f (f) (n-1)
~~~

Convince yourself that the factorial function can be defined as:

~~~
let factorial = function (n) 
              almost_factorial (almost_factorial) (n)
~~~

Eg., 

~~~
factorial (0) = almost_factorial (almost_factorial) (0) = 1

factorial (1) = almost_factorial (almost_factorial) (1) = 
              = 1 * almost_factorial (almost_factorial) (0)
              = 1 

factorial (2) = almost_factorial (almost_factorial) (2) 
              = 2 * almost_factorial (almost_factorial) (1)
              = 2 
              
         ...
~~~

This works!! Can we actually implement this and see if it works?

~~~
let almost_factorial = function (f) 
                        function (n) 
                           if (n == 0) 
                           then 1
                           else n * f (f) (n-1)
in 
let factorial = function (n) 
                  almost_factorial (almost_factorial) (n)
in 
  factorial(10)
~~~

In [ ]:
/*-- 
 Write the definition of almost_factorial
 directly into AST
 --*/

val factorial = Ident("factorial")
val almost = Ident("almost_factorial")
val n = Ident("n")
val f = Ident("f")

val e1 = FunCall(factorial, Const(10))

val double_af_call = FunDef("n", FunCall(FunCall(almost, almost), n))
val e2 = Let("factorial", double_af_call, e1)
val ite_expr = IfThenElse ( Eq(n, Const(0)), Const(1), Mult(n, FunCall(FunCall(f, f), Minus(n, Const(1)))) ) 
val  af_body = FunDef("f", FunDef("n", ite_expr))

val p10 = TopLevel( Let("almost_factorial", af_body, e2))


In [ ]:
evalProgram(p10)

In [ ]:
/*-- Try it for 8! --*/
val factorial = Ident("factorial")
val almost = Ident("almost_factorial")
val n = Ident("n")
val f = Ident("f")

val e1 = FunCall(factorial, Const(8))

val double_af_call = FunDef("n", FunCall(FunCall(almost, almost), n))
val e2 = Let("factorial", double_af_call, e1)
val ite_expr = IfThenElse ( Eq(n, Const(0)), Const(1), Mult(n, FunCall(FunCall(f, f), Minus(n, Const(1)))) ) 
val  af_body = FunDef("f", FunDef("n", ite_expr))

val p8 = TopLevel( Let("almost_factorial", af_body, e2))


In [ ]:
evalProgram(p8)

## Recipe for converting a recursive definition into non-recursive one.

Consider at a high level a function `rfun` being defined recursively as follows:

~~~
let rec rfun = function (x) <f_expr> 
  in 
  b_expr
~~~

This is recursive, therefore, in the body `f_expr`, we have recursive calls back to rfun.

- Step 1: replace all recursive calls with non recursive ones to a parameter function `f`.

~~~
let almost_rfun = function (f)  function (x) <substitute rfun in f_expr with f (f) > 
in 
... 
~~~

- Step 2: define the original function `rfun` as
~~~
let rfun = function (x) 
             almost_rfun (almost_rfun) (x)
 in 
 ...
~~~

That works and can be easily implemented in Lettuce as a function call `eliminateRecursion`.

As an example, consider

~~~
let rec crazy_rec = function (x)
                    if (x >= 0) 
                    then 0.5 * crazy_rec (1 - x)
                    else 20 * crazy_rec ( 0 - x )
 in 
 crazy_rec( - 25.0)
~~~

will be transformed into

~~~
let almost_crazy_rec = function (f) 
                          function (x) 
                             if (x >= 0) 
                             then 0.5 * f (f)  (1-x)
                             else 20 * f (f) ( 0 - x)
            in
let crazy_rec = function (x) 
                almost_crazy_rec (almost_crazy_rec) (x)
in
    crazy_rec(-25.0)
~~~

            



In [ ]:
/* Function substituteOccurrences replaces every where Identifier(oldX)
       occurs in e with the expression newX.
       We will need this function to convert recursion to non-recursive version.
       */
    
def substituteOccurrences(e: Expr, oldX: String, newX: Expr): Expr = { 
    // Define helper functions
     def applyTo (e1: Expr, e2: Expr) (res: (Expr,Expr) => Expr) = {
            res(substituteOccurrences(e1, oldX, newX), 
                substituteOccurrences(e2, oldX, newX)) 
     }
    
    def subs(e: Expr): Expr = substituteOccurrences(e, oldX, newX)
    // Now do the case matching
    e match {
   
        case Const(_) => e // Nothing to be done
        case Ident(x) => if (x == oldX) newX else e // This is where the substitution happens!!
        case Plus(e1, e2) => applyTo (e1, e2) (Plus(_, _)) 
        case Minus(e1, e2) => applyTo (e1, e2) (Minus(_, _)) 
        case Mult(e1, e2) => applyTo (e1, e2) (Mult(_, _)) 
        case Eq(e1, e2) => applyTo (e1, e2) (Eq(_, _))
        case Geq(e1, e2) => applyTo (e1, e2) (Geq(_, _))
        case IfThenElse(e, e1, e2) => IfThenElse( subs(e), subs(e1), subs(e2) )
        case Let(x, e1, e2) => {
            val e1Hat = subs(e1)
            if (x == oldX)
                Let(x, e1Hat, e2) // Do not substitute anymore since oldX is not re-bound to e1Hat and thus out of scope
            else
                Let(x, e1Hat, subs(e2))
        }
        
        case FunDef(x, e) => {
            if (x == oldX)
                FunDef(x, e) // Do not substitute since scope of oldX ends beacuse the function parameter shadows it
            else 
                FunDef(x, subs(e))
        }
        
        case FunCall(e1, e2) => {
            FunCall(subs(e1), subs(e2))
        }
        
        case LetRec(f, x, e1, e2) => {
            val e1Hat = if (x == oldX) e1 else subs(e1)
            val e2Hat = if (f == oldX) e2 else subs(e2)
            LetRec(f, x, e1Hat, e2Hat)
        }
    }
}

// Function: eliminateRecursion
//  Eliminate all let rec from expression e
// id is just used to generate fresh temporary names that we will need.

def eliminateRecursion(e: Expr, id: Int = 0): Expr = e match {    
    case Const(_) => e // Nothing to be done
    case Ident(_) => e // Nothing to be done
    case Plus(e1, e2) => Plus( eliminateRecursion(e1), eliminateRecursion(e2))
    case Minus(e1, e2) => Minus ( eliminateRecursion(e1), eliminateRecursion(e2))
    case Mult(e1, e2) => Mult( eliminateRecursion(e1), eliminateRecursion(e2))
    case Eq(e1, e2) => Eq( eliminateRecursion(e1), eliminateRecursion(e2))
    case Geq(e1, e2) => Geq(  eliminateRecursion(e1), eliminateRecursion(e2))
    case IfThenElse(e, e1, e2) => IfThenElse( eliminateRecursion(e), eliminateRecursion(e1), eliminateRecursion(e2))
    case Let(x, e1, e2) => Let(x, eliminateRecursion(e1), eliminateRecursion(e2) )
    case FunDef(x, e) => FunDef(x, eliminateRecursion(e))
    case FunCall(e1, e2) => FunCall(eliminateRecursion(e1), eliminateRecursion(e2))
    case LetRec(rfun, x, fExpr0, bExpr0) => {
        /*-- This is where action is 
            (a) substitute a new fresh string "almost_rfun" wherever rfun is referenced in fExpr
            (b) define the function call almost_rfun
            (c) define rfun as a simple function call in terms of almost_rfun
        --*/
        val fExpr = eliminateRecursion(fExpr0, id + 1)
        val bExpr = eliminateRecursion(bExpr0, id + 2)
        val almost_name = "almost_"+rfun // Name of new function -- can be more sophisticated in choosing a name
        val almost = Ident(almost_name)
        val fun_defn = Let(rfun, FunDef(x, FunCall(FunCall(almost, almost), Ident(x))), bExpr)
        val tmp_fun_name = "rfun_param_tmp__" + id.toString + "__"
        val tmp_expr = FunCall(Ident(tmp_fun_name), Ident(tmp_fun_name))
        val new_f_expr = substituteOccurrences(fExpr, rfun, tmp_expr) // Carry out the substitution
        val almost_rfun_defn = FunDef(tmp_fun_name, FunDef(x, new_f_expr))
        val almost_rfun = Let(almost_name, almost_rfun_defn, fun_defn)
        almost_rfun
    }
}




In [ ]:
val e1 = LetRec("f", "z", IfThenElse( 
                         Geq(Const(0), Ident("z")),
                         Const(1),
                         Plus(Const(1), FunCall(Ident("f"), Minus(Ident("z"), Const(1))))
                         ), 
                  FunCall(Ident("f"), Const(10))
         )
val p1 = TopLevel(e1)

val p2 = TopLevel( 
            eliminateRecursion(e1)
         )
    

In [ ]:
evalProgram(p2)

In [ ]:
val e7=  LetRec("fact", "n", 
          IfThenElse(Eq(Ident("n"), Const(0)), Const(1), 
                     Mult(Ident("n"), FunCall(Ident("fact"), Minus(Ident("n"), Const(1))))
                    ),
           FunCall(Ident("fact"), Const(7))
          )

val fact_rec_7 = TopLevel(
   e7
)

val fact_no_rec_7= TopLevel( eliminateRecursion(e7) )

In [ ]:
evalProgram(fact_no_rec_7)

# Handling Recursion using Environments

As we already saw, one strategy to handle recursion is by rewriting the recursive call to eliminate
recursion. Another approach is to  directly write an interpreter that handles recursion. However, the problem is that if we are looking at  a recursive call:

~~~
let rec f = function (x) <body of function expr> in <def. expr> 
~~~

The main problem to solve is that inside `<body of function expr>` everytime we see a reference to `f`, we should
be able to resolve it to the recursive call itself.  

Let us consider how we handle a normal non-recursive definition

~~~
let f = function (x) <body of function expr> in <def. expr>
~~~

Under enviroment $\sigma$, we evaluate the function body to a closure that stores three things: 
Closure(x,  `<body of function expr>`, $\sigma$).

This does not work for a recursive function `f` since when we execute a recursive call to `f` in the
`<body of function expr>`, it is not defined in the environment $\sigma$.

Therefore, the strategy is to extend $\sigma$ into a new environent $\color{red}{\hat{\sigma}}$ as follows:
- $\color{red}{\hat{\sigma}}(x) = \sigma(x)$ for all identifiers $x \not= f$.
- $\color{red}{\hat{\sigma}}(f) = \texttt{Closure}(x, \texttt{<body of function expr>}, \color{red}{\hat{\sigma}})$

I.e, $\color{red}{\hat{\sigma}}$ defines the function $f$ as a closure and notice that 
that the environment of the closure is $\color{red}{\hat{\sigma}}$  again! This means that if we call
$f$ in the body of the function expr, we get the same environment $\hat{\sigma}$ with just the formal parameter
$x$ updated. This gives us what we need to implement recursion: an environment that keeps defining $f$ to the
appropriate function.


Thus, we are going to reconsider environments from scratch?

## Enviroments

What is an environment? We have defined an environment as a function from identifiers to values denoted by them. What sort of environments have we encountered?
- The empty environment: implemented as an empty map. Let us call this environment: $\texttt{EmptyEnv}$.
- The environment $\sigma[x \mapsto v]$ which denotes a previously existing environment $\sigma$ extended with the mapping $x \mapsto v$ that associates identifier $x$ with value $v$. Let us call this operation 
$\texttt{Extend}(\sigma, x, v)$.

Let us add a third kind of extension to support recursion:
- $\texttt{ExtendRec}(f, x, \texttt{e}, \sigma)$ which creates a new environment $\color{red}{\hat{\sigma}}$ such that $\color{red}{\hat{\sigma}}(x) = \sigma(x)$ for all identifiers $x \not= f$ and $\color{red}{\hat{\sigma}}(f) = \texttt{Closure}(x, \texttt{<body of function expr>}, \color{red}{\hat{\sigma}})$.





In [ ]:
sealed trait Environment 
sealed trait Value

case object EmptyEnv extends Environment // Represent an environment with nothing in it.
case class Extend(x: String, v: Value, sigma: Environment) extends Environment
//ExtendRec helps me create the illusion of a circular scope
case class ExtendRec(f: String, x: String, e: Expr, sigma: Environment ) extends Environment

/* -- We need to redefine values to accomodate the new representation of environments --*/
case class NumValue(d: Double) extends Value
case class BoolValue(b: Boolean) extends Value
case class Closure(x: String, e: Expr, pi: Environment) extends Value
case object ErrorValue extends Value


/*2. Operators on values */

def valueToNumber(v: Value): Double = v match {
    case NumValue(d) => d
    case _ => throw new IllegalArgumentException(s"Error: Asking me to convert Value: $v to a number")
}

def valueToBoolean(v: Value): Boolean = v match {
    case BoolValue(b) => b
    case _ => throw new IllegalArgumentException(s"Error: Asking me to convert Value: $v to a boolean")
}

def valueToClosure(v: Value): Closure = v match {
    case Closure(x, e, pi) => Closure(x, e, pi)
    case _ =>  throw new IllegalArgumentException(s"Error: Asking me to convert Value: $v to a closure")
}


/*-- Operations on environments --*/

def lookupEnv(sigma: Environment, x: String): Value = sigma match {
    case EmptyEnv => Error // throw new IllegalArgumentException(s"Error could not find string $x in environment")
    case Extend(y, v, rest) => {
        if (y == x ) { v}
        else { lookup (rest, x)}
    } 
    // The illusion of circularity is here.
    case ExtendRec(funName, param, funBody, rest) => 
                                  if (x == funName) 
                                          Closure(param, funBody, sigma)
                                   else
                                          lookupEnv(rest, x)
}


In [ ]:
/*-- We can rewrite evalExpr now to handle recursion as well --*/
def evalExpr(e: Expr, env: Environment): Value =  {
    
    /* Method to deal with binary arithmetic operations */
    def applyArith2 (e1: Expr, e2: Expr) (fun: (Double , Double) => Double) = {
        val v1 = valueToNumber(evalExpr(e1, env))
        val v2 = valueToNumber(evalExpr(e2, env))
        val v3 = fun(v1, v2)
        NumValue(v3)
    }  /* -- We have deliberately curried the method --*/
    
    /* Helper method to deal with unary arithmetic */
    def applyArith1(e: Expr) (fun: Double => Double) = {
        val v = valueToNumber(evalExpr(e, env))
        val v1 = fun(v)
        NumValue(v1)
    }
    
    /* Helper method to deal with comparison operators */
    def applyComp(e1: Expr, e2: Expr) (fun: (Double, Double) => Boolean) = {
        val v1 = valueToNumber(evalExpr(e1, env))
        val v2 = valueToNumber(evalExpr(e2, env))
        val v3 = fun(v1, v2)
        BoolValue(v3)
    }
    
   
    e match {
        case Const(f) => NumValue(f) // Same as before
        
        case Ident(x) => lookupEnv(env, x) // Changed to accomodate the new environment definitions.
    
        /* Ditto as before */
        case Plus(e1, e2) => applyArith2 (e1, e2) ( _ + _ )
        /* Ditto as before */
        case Minus(e1, e2) => applyArith2(e1, e2) ( _ - _ )
        /* Ditto as before */
        case Mult(e1, e2) =>  applyArith2(e1, e2) (_ * _)
        /* Ditto as before */
        case Geq(e1, e2) => applyComp(e1, e2)(_ >= _)
        /* Ditto as before */
        case Eq(e1, e2) => applyComp(e1, e2)(_ == _)
        /* Ditto as before */
        case IfThenElse(e1, e2, e3) => {
            val v = evalExpr(e1, env)
            v match {
                case BoolValue(true) => evalExpr(e2, env)
                case BoolValue(false) => evalExpr(e3, env)
                case _ => throw new IllegalArgumentException(s"If-then-else condition expr: ${e1} is non-boolean -- evaluates to ${v}")
            }
        }
        /* Ditto as before */
        case Let(x, e1, e2) => {
            val v1 = evalExpr(e1, env)  // eval e1
            val env2 = Extend(x, v1, env) // create a new extended env
            evalExpr(e2, env2) // eval e2 under that.
        }
        /* Ditto as before */
        case FunDef(x, e) => {
            Closure(x, e, env) // Return a closure with the current enviroment.
        }
        /* Ditto as before */
        case FunCall(e1, e2) => {
            val v1 = evalExpr(e1, env)
            val v2 = evalExpr(e2, env)
            v1 match {
                case Closure(x, closure_ex, closed_env) => {
                    // First extend closed_env by binding x to v2
                    val new_env = Extend(x, v2, closed_env)
                    // Evaluate the body of the closure under the extended environment.
                    evalExpr(closure_ex, new_env)
                }
                case _ => throw new IllegalArgumentException(s"Function call error: expression $e1 does not evaluate to a closure")
            }
        }
        /*-- This is where we handle recursion --*/
        case LetRec(rfun, x, fExpr, bExpr) => {
            // Storing all the ingredients that are needed to create
            // a circular scope.
            val env2 = ExtendRec(rfun, x, fExpr, env)
            evalExpr(bExpr, env2)
        }
    }
}

def evalProgram(p: Program) = {
    p match { 
        case TopLevel(e) => evalExpr(e, EmptyEnv)
    }
}


In [ ]:
val fact_rec_8 = TopLevel(
    LetRec("fact", "n", 
          IfThenElse(Eq(Ident("n"), Const(0)), Const(1), 
                     Mult(Ident("n"), FunCall(Ident("fact"), Minus(Ident("n"), Const(1))))
                    ),
           FunCall(Ident("fact"), Const(8))
          )
)
evalProgram(fact_rec_8)

In [ ]:
/*--
let rec crazy_rec = function (x)
                    if (x >= 0)
                    then 0.5 * (1 - x)
                    else 20 * crazy_rec ( 0 - x )
 in 
 crazy_rec( - 25.0)
 --*/

val crec = Ident("crazy_rec")
val x = Ident("x")

val e1 = FunCall(crec, Const(-25.0))
val e21 = Mult(Const(0.5), Minus(Const(1), x))
val e22 = Mult(Const(20.0), FunCall(crec, Minus(Const(0), x)))
val e2 = LetRec("crazy_rec", "x", IfThenElse(Geq(x, Const(0)), e21, e22), e1)

val crazyProg = TopLevel(e2)

In [ ]:
evalProgram(crazyProg)